In [1]:
# the 1st line of code for contest
import pandas as pd
# df =  pd.read_csv(input(), sep='\t')

In [2]:
path = '../data/vk_download/'
df = pd.read_csv(path+'vk_main.csv', sep='\t')
df.head()

,city,country,first_name,home_town,in_hse_memes_group,is_bmm,is_closed,last_name,likes_memes,uid,...,photo_month_mean,photo_repost_cnt,photo_repost_max,photo_repost_mean,photo_repost_median,photo_text_len_cnt,photo_ava_change_cnt,photo_text_url_len_cnt,friends_from_course_cnt,friends_mail_from_course_pct
0,Москва,Россия,Александра,Москва,True,True,False,Абашкова,60.0,182152789,...,1.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,42.0,0.428571
1,Рязань,Россия,Анастасия,Рязань,True,True,False,Чуфистова,0.0,148020433,...,2.375000,2.0,1.0,0.105263,0.0,0.0,0.0,0.0,32.0,0.281250
2,Москва,Россия,Александр,Омск,False,True,False,Головачев,0.0,138413935,...,1.400000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,32.0,0.406250
3,NaN,NaN,Анна,NaN,False,True,False,Лобанова,0.0,366261055,...,4.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,45.0,0.333333
4,NaN,Россия,Алексей,NaN,True,True,False,Пузырный,21.0,111252392,...,3.181818,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,41.0,0.341463


In [3]:
# 1

print(sum((df['is_bmm'] == 1) & (df['has_ava_dummy'] == 0)))


6


In [4]:
# 2

gr = df.groupby(['male_dummy','instagram_dummy'])[['english_dummy']].mean()
gr = gr.sort_index(level=[0,1],ascending=[True,True])

print('\n'.join(['%0.6f' % i[0] for i in gr.values]))

0.126437
0.214286
0.142857
0.185185


In [5]:
# 3

df.dropna(subset=['wall_emoji_trace'],inplace=True)
df['wall_emoji_trace'] = df['wall_emoji_trace'].astype(str).apply(len)

p = df[df['wall_emoji_trace']>=df['wall_emoji_trace'].quantile(0.95)]
p = p.sort_values(['wall_emoji_trace','first_name'],ascending=[False,True])

print('\n'.join([' '.join(p.iloc[i][['first_name','last_name']].tolist()) for i in range(p.shape[0])]))

Анастасия Чуфистова
Григорий Черноусов
Юлия Иванова
Elizabeth Emelyanova
Анжелика Искандерова
Алик Атаханов
Оля Федотова
Дуся Шурмистова
Аня Логинова
Стася Кокеткина
Иван Чумачков


In [6]:
# manual way to create segment col
###############################################################
q1 = df['friends_from_course_cnt'].quantile(0.25)
q2 = df['friends_from_course_cnt'].quantile(0.5)
q3 = df['friends_from_course_cnt'].quantile(0.75)

df['segment'] = df['friends_from_course_cnt'].apply(
    lambda x: (x<=q1)*1 + (q1<x<=q2)*2 + (q2<x<=q3)*3 + (q3<x)*4)
###############################################################

df[['friends_from_course_cnt','segment']].head(10)


,friends_from_course_cnt,segment
1,32.0,3
2,32.0,3
3,45.0,4
4,41.0,3
6,31.0,3
7,32.0,3
8,47.0,4
10,24.0,2
12,25.0,2
13,21.0,1


In [7]:
def get_answers(df):
    # automatic way to create segment col
    df['segment'] = 1+pd.qcut(df['friends_from_course_cnt'],q=[0, .25, .5, .75, 1.],labels=False)
    gr = df.groupby(['is_bmm'])['friends_from_course_cnt'].median()
    a1 = gr.loc[True] > gr.loc[False]
    print(a1)

    gr = df.groupby(['is_bmm','segment'])['friends_from_course_cnt'].count()
    a2 = gr.loc[True,4] > gr.loc[False,4]
    print(a2)
    print(a1==a2)

    gr = df.groupby('segment')['likes_memes'].mean()
    a4 = gr.loc[1] < gr.loc[2] < gr.loc[3] < gr.loc[4]
    print(a4)
    

    
df_orig = df.copy()


df2 = df_orig.copy()
df2.dropna(subset=['friends_from_course_cnt'],inplace=True)
get_answers(df2)

df2 = df_orig.copy()
df2['friends_from_course_cnt'].fillna(0,inplace=True)
get_answers(df2)

df2 = df_orig.copy()
df2['friends_from_course_cnt'].fillna(df['friends_from_course_cnt'].mean(),inplace=True)
get_answers(df2)

df2 = df_orig.copy()
df2['friends_from_course_cnt'].fillna(df['friends_from_course_cnt'].median(),inplace=True)
get_answers(df2)
        

False
False
True
False
False
False
True
False
False
False
True
False
False
False
True
False
